In [28]:
import os
import requests
import csv
import linecache
import numpy as np
import pandas as pd

In [29]:
#change these
filename = "22-08rw"
html_ = f"{filename}.html"
csv_ =  f"{filename}.csv"


In [30]:
# Path
data_folder = "data"
 
# Join various path components
html_file = os.path.join(data_folder, html_)
print(html_file)

data\22-08rw.html


In [31]:
#get meet name, usually on line 6, print to doublecheck
meet = linecache.getline(html_file, 6).strip()
print(meet)

2021 PV RMSC October Kick-Off Classic - 10/15/2021 to 10/17/2021


In [32]:
# #male or female to be added to the csv
# #not needed anymore as we pull this from event
# if filename[-1] == "m":
#     gender = "man"
# elif filename[-1] == "w":
#     gender = "woman"
# else: "unknown"

print(gender)

Boys


In [33]:
# event = "Girls 13 Over 50 Yard Freestyle"
# temp_string = event.replace(" Over", "Over").split(" ")
# gender = temp_string[0]
# age_group = temp_string[1]
# distance = temp_string[2]
# scy_lcy = temp_string[3]
# stroke = temp_string[-1]
# print(temp_string)

In [34]:
#Creating an output file in writing mode
csv_file = os.path.join(data_folder,csv_)

#csv header
header = ["meet", "event", "gender", "age_group", "distance", "scy_lcy", "stroke","rank", "name", "age", "team", "seed", "finals"]
#assign 'delete' event for delition later
event = "delete"

with open(csv_file, 'w', encoding='UTF8') as f:
    #create a writer
    writer = csv.writer(f, delimiter = "|")
    # write the header
    writer.writerow(header)
    #iterate html
    with open(html_file, "r") as line:
        line_string = line.readlines()
        #this will assign event column
        for each_line in line_string:
            if each_line[0:2] == "<b":
                event = each_line.replace("<b>", "")\
                                 .replace("</b>", "")\
                                 .replace("&amp;", "&")\
                                 .replace("\n", " ")\
                                 .rstrip()
                #temp_string converts event to string
                #replacing space in 13 and over to standardize list length
                temp_string = event.replace(" Over", "Over").split(" ")
                gender = temp_string[0]
                age_group = temp_string[1]
                distance = temp_string[2]
                scy_lcy = temp_string[3]
                stroke = temp_string[-1]
            else:
                #assign each column and strip right spaces
                rank = each_line[0:4].replace(" ", "").replace("=", "").rstrip()
                name = each_line[4:29].rstrip()
                age = each_line[28:33].replace(" ","").rstrip()
                team = each_line[33:50].replace(" ","").rstrip()
                #replacing NT to very high number
                #also replacing DQ qith very low number
                seed = each_line[51:65].replace(" ","").replace("X","").\
                    replace("NT","10000.00").replace("DQ","00.00").rstrip()
                finals = each_line[61:80].replace(" ","").replace("X","").\
                    replace("NT","10000.00").replace("DQ","00.00").rstrip()
                #this will not write uncessary rows
                if rank != "" and event != 'delete'\
                   and seed != "" and finals != "":
                    writer.writerow([meet, event, gender, age_group, distance, scy_lcy, stroke, rank, name, age, team, seed, finals])



In [35]:
#read csv to data frame
df = pd.read_csv(csv_file, sep="|"  , engine="python")

In [36]:
#craete function to convert time string to float
def f_totime(i):
    #find minute indicator
    minute_ind = i.find(':')
    if (minute_ind != -1):
        i = float(i[0:minute_ind]) * 60 + float(i[minute_ind:].replace(":", ""))
        return i
    else:
        return i

In [37]:
#create new column from seed
df["seed_time"] = df.apply(lambda row: f_totime(row["seed"]),axis=1)
#create new column from final
df["finals_time"] = df.apply(lambda row: f_totime(row["finals"]),axis=1)

In [38]:
#change data type for seed time and final time
df["finals_time"] = df["finals_time"].astype(np.float16)

df["seed_time"] = df["seed_time"].astype(np.float16)

#create delta column
df["delta"] = df["finals_time"] - df["seed_time"]


In [39]:
#function converting delta gain/drop/same
def f_gain_drop(i):
    if i < 0:
        return "Drop"
    elif i > 0:
        return "Gain"
    else:
        return "Same"

In [40]:
#create new column for drop/gain/same
df["gain_drop_same"] = df.apply(lambda row: f_gain_drop(row["delta"]),axis=1)

In [41]:
#export to csv, with pipe separator
#and donot write when blank
with open(csv_file, 'w', newline="") as csv_file:
    df.to_csv(path_or_buf=csv_file, sep='|', index=False)